In [1]:
import time
from utils.data_compression import decompress_data_blosc

import time
import sys

data_store = {}
import matplotlib.pyplot as plt
import numpy as np


def get_total_size(obj, seen=None):
    """Recursively calculates the size of objects in bytes, including nested objects."""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
        
    obj_id = id(obj)
    if obj_id in seen:
        return 0

    # Mark this object as seen
    seen.add(obj_id)

    if isinstance(obj, dict):
        size += sum([get_total_size(v, seen) for v in obj.values()])
        size += sum([get_total_size(k, seen) for k in obj.keys()])
    elif isinstance(obj, list) or isinstance(obj, tuple) or isinstance(obj, set):
        size += sum([get_total_size(i, seen) for i in obj])
    
    return size

def get_total_size_in_mb(obj):
    """Returns the total size of an object in megabytes."""
    size_in_bytes = get_total_size(obj)
    size_in_mb = size_in_bytes / (1024 * 1024)  # Convert bytes to MB
    return size_in_mb

def display_capabilities_table(mpClient, max_width=75):
    """
    Retrieves and displays capabilities in a table format, wrapping long fields line-by-line.
    
    Parameters:
    - mpClient: Client instance to retrieve capabilities.
    - capability_type: Type of capability to filter by (default is "measure").
    - max_width: Maximum width per line for text fields to wrap.
    """
    capabilities = mpClient.get_capabilities()

    if capabilities:
        # Print table headers
        print(f"{'ID':<5} {'Endpoint':<20} {'Label':<40} {'Parameters Schema':<80} {'Result Schema':<80}")
        print("-" * 210)

        # Helper function to wrap text
        def wrap_text(text, width):
            return [text[i:i + width] for i in range(0, len(text), width)]
        
        # Print each capability row
        for c_id, cap in capabilities.items():
            # Get values and wrap long text fields
            endpoint_lines = wrap_text(str(cap.get('endpoint', 'N/A')), max_width)
            label_lines = wrap_text(str(cap.get('label', 'N/A')), max_width)
            parameters_schema_lines = wrap_text(str(cap.get('parameters_schema', 'N/A')), max_width)
            results_schema_lines = wrap_text(str(cap.get('resultSchema', 'N/A')), max_width)
            
            # Find maximum number of lines among the fields to align the row height
            max_lines = max(len(endpoint_lines), len(label_lines), len(parameters_schema_lines), len(results_schema_lines))
            
            # Print each line of the wrapped text fields
            for i in range(max_lines):
                c_id_display = str(c_id) if i == 0 else ""  # Display ID only in the first line
                endpoint_display = endpoint_lines[i] if i < len(endpoint_lines) else ""
                label_display = label_lines[i] if i < len(label_lines) else ""
                parameters_display = parameters_schema_lines[i] if i < len(parameters_schema_lines) else ""
                results_display = results_schema_lines[i] if i < len(results_schema_lines) else ""
                
                # Print each row with formatted spacing
                print(f"{c_id_display:<5} {endpoint_display:<20} {label_display:<40} {parameters_display:<80} {results_display:<80}")
            
            # Add a separator line after each entry for readability
            print("-" * 210)
    else:
        print("No available capabilities")
    return capabilities



def on_result_callback(result):
    Dresult_size = 0
    for ch in result[0]:
        if ch not in data_store:
            data_store[ch] = {}
        print("received for channel: ",ch)
        for wr_time in result[0][ch]:
            data_store[ch][wr_time] = decompress_data_blosc(result[0][ch][wr_time])
            print("WR_Time: ",wr_time)
            print("timetags number: ", len(data_store[ch][wr_time]))
            Dresult_size += get_total_size_in_mb(data_store[ch][wr_time])
            print("data is:" , data_store[ch][wr_time])
    result_size =  get_total_size_in_mb(result[0])
    print(f"++++++++Total size of result: {result_size:.2f} MB")
    print(f"++++++++Total size of dresult: {Dresult_size:.2f} MB")
    
    pass


In [14]:
from measurement_plane.measurement_plane_client.MP_client import MeasurementPlaneClient
from utils.data_compression import decompress_data_blosc
data_store = {}
def on_result_callback(result):
    #d_result = decompress_data_blosc(result[0])
    #print("++++++++++++++data is:" , d_result)
    for ch in result[0]:
        if ch not in data_store:
            data_store[ch] = {}
        print("received for channel: ",ch)
        for wr_time in result[0][ch]:
            data_store[ch][wr_time] = decompress_data_blosc(result[0][ch][wr_time])
            print("WR_Time: ",wr_time)
            print("timetags number: ", len(data_store[ch][wr_time]))
            print("data is:" , data_store[ch][wr_time])
    pass

BROKER_URL = "http://localhost:5672/"

mpClient = MeasurementPlaneClient(BROKER_URL)

In [3]:
# Example usage:
capabilities = display_capabilities_table(mpClient)

ID    Endpoint             Label                                    Parameters Schema                                                                Result Schema                                                                   
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
9c56509c87191d5cb5bd19f856be4860aeb7ab8bc3de3df01838a0df41e057ff /tt/Alice            time tags measurement capability         {'type': 'object', 'properties': {'channels': {'type': 'string', 'descripti      {'type': 'object', 'patternProperties': {'^[0-9]+$': {'type': 'object', 'pr     
                                                                    on': "String of channel numbers separated by '|' to measure timetags from"}      operties': {'WR_time': {'type': 'array', 'items': {'type': 'string'}}}, 're     
                                        

In [4]:
# Step 2: Chose the desired capability by it's id
capability_id = "9c56509c87191d5cb5bd19f856be4860aeb7ab8bc3de3df01838a0df41e057ff"
capability = capabilities[capability_id]

In [15]:
# Step 3: Instantiate a specification of a measurement from the capability
measurement = mpClient.create_measurement(capability)
print(f"Created Measurement: {measurement.specification_message}")

# Step 4: Configure the measurement with parameters
channels =  '2'
schedule = "now ||stream"
parameters = {
    "channels": channels
}
measurement.configure(
    schedule=schedule,
    parameters=parameters,
    result_callback=on_result_callback,  # Callback function for new results
)
print(f"Configured Measurement Specification: {measurement.specification_message}")

# Step 5: Send the Measurement
mpClient.send_measurement(measurement)

Created Measurement: {'label': 'time tags measurement capability', 'endpoint': '/tt/Alice', 'capabilityName': 'Time_tags_measurement_capability', 'parameters_schema': {'type': 'object', 'properties': {'channels': {'type': 'string', 'description': "String of channel numbers separated by '|' to measure timetags from"}}}, 'resultSchema': {'type': 'object', 'patternProperties': {'^[0-9]+$': {'type': 'object', 'properties': {'WR_time': {'type': 'array', 'items': {'type': 'string'}}}, 'required': ['WR_time']}}}, 'timestamp': '2024-12-14 23:18:55.41', 'nonce': '12345', 'metadata': None, 'specification': 'measure-time-tags'}
Configured Measurement Specification: {'label': 'time tags measurement capability', 'endpoint': '/tt/Alice', 'capabilityName': 'Time_tags_measurement_capability', 'parameters_schema': {'type': 'object', 'properties': {'channels': {'type': 'string', 'description': "String of channel numbers separated by '|' to measure timetags from"}}}, 'resultSchema': {'type': 'object', 'p

received for channel:  2
WR_Time:  1488638.0
timetags number:  999987
data is: [48111757670539 48111758670554 48111759670768 ... 49111814093300
 49111815093467 49111816093446]
received for channel:  2
WR_Time:  1488639.0
timetags number:  999988
data is: [1.56580643e+04 1.01556231e+06 2.01551555e+06 ... 9.99997670e+11
 9.99998670e+11 9.99999670e+11]
received for channel:  2
WR_Time:  1488640.0
timetags number:  999987
data is: [6.70512930e+05 1.67054517e+06 2.67054441e+06 ... 9.99997327e+11
 9.99998327e+11 9.99999327e+11]
received for channel:  2
WR_Time:  1488641.0
timetags number:  999988
data is: [3.26992459e+05 1.32698470e+06 2.32702494e+06 ... 9.99997992e+11
 9.99998992e+11 9.99999992e+11]
received for channel:  2
WR_Time:  1488642.0
timetags number:  999987
data is: [9.92868667e+05 1.99277891e+06 2.99280415e+06 ... 9.99997658e+11
 9.99998658e+11 9.99999658e+11]
received for channel:  2
WR_Time:  1488643.0
timetags number:  999987
data is: [6.58229665e+05 1.65819891e+06 2.65827714

In [16]:
# Step 6: Interrupt the measurement
mpClient.interrupt_measurement(measurement)
print(f"Measurement Interrupted")

will close the receiver
Measurement Interrupted


In [ ]:
data_store
import pickle
with open('data_store.pkl', 'wb') as file:
    pickle.dump(data_store, file)


In [ ]:
import pickle
# Load the data
with open('data_store.pkl', 'rb') as file:
    retreived_data = pickle.load(file)

In [ ]:
retreived_data
len(retreived_data[2])

In [ ]:
retreived_data[2]


In [ ]:
def plot_counts(retreived_data):
    plt.figure()
    plt.xlabel("Absolute Time (1PPS)")
    plt.ylabel("Count of Events")
    plt.title("Counts for Each Channel")
    plt.ylim(0, 3.1e6)
    # Iterate through each channel and add a bar plot for its counts
    for channel, time_data in data_store.items():
        counts = []
        abs_times = sorted(time_data.keys())
        # Calculate counts for each absolute time
        for abs_time in abs_times:
            counts.append(len(time_data[abs_time]))
        # Plot counts for the channel on the same figure with a label
        plt.plot(abs_times, counts, marker='o', linestyle='-',label=f"Channel {channel}")
    
    # Show legend to differentiate channels
    plt.legend()
    plt.show()

plot_counts(retreived_data)
